In [93]:
import numpy as np
import pandas as pd

#for getting world map
import folium

### Data Collection

In [94]:
# To retrieve Latitude and Longitude coordinates use read_html() method
# read_html() method returns all the tables in the webpage (Internally beautifulSoup is executed)
info = pd.read_html("http://www.quickgs.com/latitudinal-and-longitudinal-extents-of-india-indian-states-and-cities/")

# convert the table into DataFrame
coordinates = pd.DataFrame(info[0])
coordinates.head()

,State,Latitude,Longitude
0,Andhra Pradesh,15.91°N,79.74°E
1,Arunachal Pradesh,28.21°N,94.72°E
2,Assam,26.20°N,92.93°E
3,Bihar,25.09°N,85.31°E
4,Chhattisgarh,21.27°N,81.86°E


In [95]:
# Retrevie the LIVE COVID19 Stats from Wikipedia
corona_stats = pd.read_html('https://en.wikipedia.org/wiki/COVID-19_pandemic_in_India#covid19-container',match='State/Union Territory')
covid_cases= pd.DataFrame(corona_stats[0])
covid_cases.head()

[show all]viewtalkeditCOVID-19 pandemic in India by state and union territory  \
                                                          State/Union Territory   
                                                                        35 / 36   
0                        Andaman and Nicobar Islands                              
1                                     Andhra Pradesh                              
2                                  Arunachal Pradesh                              
3                                              Assam                              
4                                              Bihar                              

                                       
   Cases[a] Deaths Recoveries  Active  
    366,946 12,237    194,325 160,384  
0        44      0         33      11  
1      7071     90       3641    3340  
2        99      0          7      92  
3  4,605[b]      8       2642    1955  
4      6942     44       4926    1972

### Data Cleaning and Preparation

In [96]:
# Data cleaning,Transforming Latitude and Longitude into algorithm compatible form
# Removing degree symbol in lattitudes and longitudes and changing their data type
def remove_degree_symbol(coordinate):
    return str(coordinate)[0:5]
coordinates['Latitude'] = coordinates['Latitude'].apply(remove_degree_symbol).astype('float')
coordinates['Longitude'] = coordinates['Longitude'].apply(remove_degree_symbol).astype('float')
coordinates.head()

,State,Latitude,Longitude
0,Andhra Pradesh,15.91,79.74
1,Arunachal Pradesh,28.21,94.72
2,Assam,26.20,92.93
3,Bihar,25.09,85.31
4,Chhattisgarh,21.27,81.86


In [98]:
# Cleaning covid_cases Dataframe
# Renaming Attribute Names for simplicity
covid_cases.columns = ['State','Total Cases','Deaths','Recoveries','Active Cases']
# Removing unnecessary rows at the tail
covid_cases = covid_cases.iloc[:-3,:]
covid_cases.head()

,State,Total Cases,Deaths,Recoveries,Active Cases
0,Andaman and Nicobar Islands,44,0,33,11
1,Andhra Pradesh,7071,90,3641,3340
2,Arunachal Pradesh,99,0,7,92
3,Assam,"4,605[b]",8,2642,1955
4,Bihar,6942,44,4926,1972


In [99]:

#Merging the tables covid_cases and coordinates 
final_data = pd.merge(coordinates, covid_cases, how ='inner', on ='State')
final_data.head()

,State,Latitude,Longitude,Total Cases,Deaths,Recoveries,Active Cases
0,Andhra Pradesh,15.91,79.74,7071,90,3641,3340
1,Arunachal Pradesh,28.21,94.72,99,0,7,92
2,Assam,26.20,92.93,"4,605[b]",8,2642,1955
3,Bihar,25.09,85.31,6942,44,4926,1972
4,Chhattisgarh,21.27,81.86,1864,10,1180,674


### Visualization

In [100]:
# retreiving the data from final table and plotting it on the INDIA map
# creating the map object zooming on INDIA, location here shows the lat and long of INDIA
india = folium.Map(location = [20.5937,78.9629],zoom_start=4.5)

#adding to map
for state,lat,long,total_cases,Death,Recov,Active in zip(list(final_data['State']),list(final_data['Latitude']),
                                                         list(final_data['Longitude']),list(final_data['Total Cases']),
                                                         list(final_data['Deaths']),list(final_data['Recoveries']),
                                                         list(final_data['Active Cases'])):
    #for creating circle marker
    folium.CircleMarker(location = [lat,long],
                       radius = 5,
                       color='red',
                       fill = True,
                       fill_color="red").add_to(india)
    #for creating marker
    folium.Marker(location = [lat,long],
                  # adding information that need to be displayed on popup
                  popup=folium.Popup(('<strong><b>State  : '+state+'</strong> <br>' +
                    '<strong><b>Total Cases : '+total_cases+'</striong><br>' +
                    '<strong><font color= red>Deaths : </font>'+Death+'</striong><br>' +
                    '<strong><font color=green>Recoveries : </font>'+Recov+'</striong><br>' +
                    '<strong><b>Active Cases : '+Active+'</striong>' ),max_width=200)).add_to(india)
#to show the map
display(india)

## Covid19 Visualization of Austria

In [101]:
# Coordinates of the states
coor_info = pd.read_html("https://www.distancelatlong.com/country/austria")
coordinates = pd.DataFrame(coor_info[2])
coordinates['States'] = coordinates['States'].apply(lambda x: x[:-4].strip())
coordinates.rename(columns={'States':'State'}, inplace=True)

# confirmed cases
case_info = pd.read_html('https://en.wikipedia.org/wiki/COVID-19_pandemic_in_Austria')
austria_cases = pd.DataFrame(case_info[3])
austria_cases.drop(austria_cases.columns[[0,4,5]], axis=1, inplace=True)
austria_cases = austria_cases.iloc[:-1,:]
austria_cases['State'] = coordinates['State']

# merge to obtain final data frame
final_data = pd.merge(coordinates, austria_cases, how ='inner', on ='State')
final_data

,State,Latitude,Longitude,Total Cases,Active,Deaths,Recov.
0,Burgenland,47.833299,16.533297,343,3,11,329
1,Karnten,46.620344,14.310020,419,4,13,402
2,Niederosterreich,47.815982,16.249954,2926,72,100,2754
3,Oberosterreich,48.319233,14.288781,2331,35,61,2235
4,Salzburg,47.810478,13.040020,1243,4,38,1201
5,Steiermark,47.077758,15.410005,1843,4,151,1688
6,Tirol,47.280407,11.409991,3554,6,108,3440
7,Vorarlberg,47.516697,9.766702,910,9,19,882
8,Wien,48.200015,16.366639,3634,280,186,3168


In [102]:
austria = folium.Map(location = [47.5162,14.5501],zoom_start=6)
for state,lat,long,total_cases,Death,Recov,Active in zip(list(final_data['State']),list(final_data['Latitude']),
                                                         list(final_data['Longitude']),list(final_data['Total Cases']),
                                                         list(final_data['Deaths']),list(final_data['Recov.']),
                                                         list(final_data['Active'])):
    #for creating circle marker
    folium.CircleMarker(location = [lat,long],
                       radius = 5,
                       color='red',
                       fill = True,
                       fill_color="red").add_to(austria)
    #for creating marker
    folium.Marker(location = [lat,long],
                  # adding information that need to be displayed on popup
                  popup=folium.Popup(('<strong><b>State  : '+state+'</strong> <br>' +
                    '<strong><b>Total Cases : '+str(total_cases)+'</striong><br>' +
                    '<strong><font color= red>Deaths : </font>'+str(Death)+'</striong><br>' +
                    '<strong><font color=green>Recoveries : </font>'+str(Recov)+'</striong><br>' +
                    '<strong><b>Active Cases : '+str(Active)+'</striong>' ),max_width=200)).add_to(austria)
display(austria)

## Covid19 Visualization of Turkey

In [103]:
coor_info = pd.read_json("https://gist.githubusercontent.com/ozdemirburak/4821a26db048cc0972c1beee48a408de/raw/4754e5f9d09dade2e6c461d7e960e13ef38eaa88/cities_of_turkey.json")
coor_info = coor_info.drop(coor_info.columns[[0,5]], axis=1)
coor_info.rename(columns={'name':'city'}, inplace=True)
coor_info['city'] = coor_info['city'].map(lambda x: 'I' + x[1:] if x.startswith('İ') else x)
coor_info.head()

,city,latitude,longitude,population
0,Adana,37.0000,35.3213,2183167
1,Adıyaman,37.7648,38.2786,602774
2,Afyonkarahisar,38.7507,30.5567,709015
3,Ağrı,39.7191,43.0503,547210
4,Amasya,40.6499,35.8353,322167


In [105]:
covid_info = pd.read_html('https://www.bbc.com/turkce/haberler-turkiye-51576487')
turkey_cases = pd.DataFrame(covid_info[0])
turkey_cases.rename(columns={'İl':'city', 'Vaka Sayısı':'cases'}, inplace=True)
turkey_cases['city'] = turkey_cases['city'].map(lambda x: 'I' + x[1:] if x.startswith('İ') else x)
turkey_cases.head()

,city,cases
0,Istanbul,28014
1,Izmir,2884
2,Ankara,2032
3,Kocaeli,1702
4,Konya,1690


In [107]:
turkey_data = pd.merge(coor_info, turkey_cases, how ='inner', on ='city')
turkey_data.head()

,city,latitude,longitude,population,cases
0,Ankara,39.9208,32.8541,5270575,2032
1,Bursa,40.2669,29.0634,2842547,1545
2,Isparta,37.7648,30.5566,421766,602
3,Istanbul,41.0053,28.9770,14657434,28014
4,Izmir,38.4189,27.1287,4168415,2884


In [108]:
turkey = folium.Map(location = [38.9637,35.2433],zoom_start=5)
# adding to map
for city,lat,long,total_cases,population in zip(list(turkey_data['city']),list(turkey_data['latitude']),list(turkey_data['longitude']),
                                      list(turkey_data['cases']),list(turkey_data['population'])):
    #for creating circle marker
    folium.CircleMarker(location = [lat,long],
                       radius = 5,
                       color='red',
                       fill = True,
                       fill_color="red").add_to(turkey)
    #for creating marker
    folium.Marker(location = [lat,long],
                  # adding information that need to be displayed on popup
                  popup=folium.Popup(('<strong><b>City  : '+ city +'</strong> <br>' +
                    '<strong><font color= red>Total Cases : </font>'+str(total_cases)+'</strong><br>' +
                    '<strong><b>Population : '+ str(population)+'</strong><br>'),max_width=200)).add_to(turkey)
# to show the map
display(turkey)